# 3. Calcolo di Metriche Avanzate e Normalizzazione

**Obiettivo:** Arricchire il dataset con metriche che mi diano una visione più profonda e comparabile delle performance, superando i limiti delle statistiche tradizionali.

1.  **Caricamento Dati:** Leggo il DataFrame pulito.
2.  **Normalizzazione 'Per 36 Minuti':** Rendo le statistiche comparabili tra giocatori con minutaggi diversi.
3.  **Calcolo del True Shooting Percentage (TS%):** Calcolo una metrica avanzata per misurare l'efficienza al tiro.
4.  **Salvataggio:** Salvo il DataFrame arricchito, pronto per il clustering.

In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.config.spark_config import SPARK_CONFIG
from src.data_processing.normalization import per_36_minutes_stats
from src.feature_engineering.advanced_metrics import calculate_true_shooting_percentage
from src.utils.helpers import get_spark_session, save_dataframe

project_root = os.path.abspath(os.path.join('..'))
processed_data_dir = os.path.join(project_root, "data", "processed")
cleaned_df_path = os.path.join(processed_data_dir, "players_cleaned.parquet")
output_path = os.path.join(processed_data_dir, "players_advanced_metrics.parquet")

spark = get_spark_session(
    app_name="NBA_Advanced_Metrics",
    driver_memory=SPARK_CONFIG["driver_memory"]
)

### Fase 1: Caricamento dei Dati Puliti

Inizio caricando il dataset pulito.

In [ ]:
df = spark.read.parquet(cleaned_df_path)

### Fase 2: Normalizzazione delle Statistiche (Per 36 Minuti)

Confrontare le statistiche totali di un titolare con quelle di una riserva è fuorviante. Normalizzando "per 36 minuti", posso confrontare il loro *impatto per minuto*, che è molto più interessante.

In [ ]:
stats_to_normalize = ['pts', 'trb', 'ast', 'stl', 'blk', 'tov', 'fga', 'fta', 'fg', 'ft']

df_normalized = per_36_minutes_stats(df, stats_to_normalize, minutes_played_col="mp")

print("Esempio di statistiche normalizzate per 36 minuti:")
df_normalized.select('player', 'season', 'pts_per_36_min', 'trb_per_36_min', 'ast_per_36_min').show(10)

#### Interpretazione dell'output

La tabella ora ha le nuove colonne `_per_36_min`. Questi valori mi danno una base di confronto molto più equa, che sarà fondamentale per il clustering.

### Fase 3: Calcolo Metriche Avanzate (True Shooting %)

Il True Shooting Percentage (TS%) è una misura di efficienza superiore alla semplice percentuale dal campo, perché tiene conto del valore dei tiri da tre e dei tiri liberi. Un TS% alto indica un giocatore molto efficiente.

In [ ]:
df_advanced = calculate_true_shooting_percentage(df_normalized, points_col="pts", fga_col="fga", fta_col="fta")

print("Top 10 Giocatori per True Shooting Percentage (calcolato):")
df_advanced.select('player', 'season', 'pts_per_36_min', 'ts_pct_calc') \
             .orderBy(df_advanced.ts_pct_calc.desc()) \
             .show(10)

#### Interpretazione dell'output

In cima a questa classifica non ci sono solo i migliori marcatori, ma giocatori (spesso lunghi) che tirano con percentuali altissime da vicino. Questa metrica mi aiuterà a distinguere i realizzatori "di volume" da quelli "efficienti".

### Fase 4: Salvataggio del DataFrame Arricchito

Salvo il DataFrame, che ora contiene sia le statistiche normalizzate che le metriche avanzate. È pronto per il clustering.

In [ ]:
save_dataframe(df_advanced, output_path)

print(f"Dati con metriche avanzate salvati con successo in '{output_path}'.")

### Conclusione e Prossimi Passi

Ho trasformato i dati grezzi in un set di feature ricco e significativo. Ora ho tutto quello che mi serve per la fase successiva.

Nel prossimo notebook, `04_player_clustering.ipynb`, userò queste feature per addestrare il modello di K-Means e raggruppare i giocatori in base al loro stile di gioco.

In [ ]:
spark.stop()